In [6]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Merge
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

In [7]:
X_train = np.load("./data/X_train_no_padding.npy")
X_test = np.load("./data/X_train_no_padding.npy")

X_validation = np.load("./data/X_validation.npy")

In [8]:
max_seq_length = 30

In [9]:
# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [2]:
X_train = np.load("./data/X_train.pkl.npy")
X_test = np.load("./data/X_train.pkl.npy")

X_validation = np.load("./data/X_validation.pkl.npy")

In [5]:
X_train

array({'left': array([[    0,     0,     0, ...,    96,   315,   782],
       [    0,     0,     0, ...,  5036,   108,  1539],
       [    0,     0,     0, ...,  3050,  2857,  1157],
       ..., 
       [    0,     0,     0, ...,   141,   135,   512],
       [    0,     0,     0, ...,     3,  4540, 32165],
       [    0,     0,     0, ...,  7527,     8,  1563]], dtype=int32), 'right': array([[    0,     0,     0, ...,   198,  5434,  3950],
       [    0,     0,     0, ...,     8, 39790,  5036],
       [    0,     0,     0, ...,     5,  1251, 13041],
       ..., 
       [    0,     0,     0, ...,    27,   512,   232],
       [    0,     0,     0, ...,     3,  4540, 32166],
       [    0,     0,     0, ...,     2, 56898,  7527]], dtype=int32)}, dtype=object)

In [3]:
# TEST
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
#n_hidden = 50
n_hidden = 10
gradient_clipping_norm = 1.25
#batch_size = 64
batch_size = 1
#n_epoch = 25
n_epoch = 10

In [ ]:
# MaLSTM similarity function
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [ ]:
# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

In [ ]:
# Calculates the distance as defined by the MaLSTM model
malstm_distance = Merge(mode=lambda x: exponent_neg_manhattan_distance(x[0], x[1]), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

In [ ]:
# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

In [ ]:
# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, epochs=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation), validation_split = 0.2)

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

In [ ]:
# Plot accuracy
plt.plot(malstm_trained.history['acc'])
plt.plot(malstm_trained.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()